In [1]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import  DataLoader,Dataset
from torchvision.transforms import ToTensor

In [3]:
train_data=datasets.FashionMNIST(root="data",train=True,download=True,transform=ToTensor())
test_data=datasets.FashionMNIST(root="data",train=False,download=True,transform=ToTensor())

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
train_dataLoader=DataLoader(train_data,batch_size=64)
test_dataLoader=DataLoader(test_data,batch_size=64)


In [6]:
for X,y in train_dataLoader:
  print(X.shape)
  print(y.shape)
  print(y.dtype)
  break

torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.int64


In [9]:
device="cuda" if torch.cuda.is_available() else "cpu"
from torch.nn.modules import flatten
from torch.nn.modules.linear import Linear
from torch.nn.modules.activation import ReLU
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten=nn.Flatten()
    self.linear_relu_stack=nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)       
    )
  def forward(self,x):
    x=self.flatten(x)
    logits=self.linear_relu_stack(x)
    return logits

model=NeuralNetwork().to(device)
print(model)



NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.001)

In [12]:
def train(dataloader,model,loss_fn,optimizer):
  size=len(dataloader.dataset)
  model.train()
  for batch,(X,y) in enumerate(dataloader):
    X,y=X.to(device),y.to(device)
    pred=model(X)
    loss=loss_fn(pred,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [13]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataLoader, model, loss_fn, optimizer)
    test(test_dataLoader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.290099  [    0/60000]
loss: 2.288680  [ 6400/60000]
loss: 2.267927  [12800/60000]
loss: 2.267579  [19200/60000]
loss: 2.255155  [25600/60000]
loss: 2.216183  [32000/60000]
loss: 2.230251  [38400/60000]
loss: 2.195587  [44800/60000]
loss: 2.191876  [51200/60000]
loss: 2.156332  [57600/60000]
Test Error: 
 Accuracy: 39.0%, Avg loss: 2.159151 

Epoch 2
-------------------------------
loss: 2.162096  [    0/60000]
loss: 2.157562  [ 6400/60000]
loss: 2.103462  [12800/60000]
loss: 2.118395  [19200/60000]
loss: 2.080975  [25600/60000]
loss: 2.011497  [32000/60000]
loss: 2.039371  [38400/60000]
loss: 1.968936  [44800/60000]
loss: 1.966609  [51200/60000]
loss: 1.893208  [57600/60000]
Test Error: 
 Accuracy: 58.6%, Avg loss: 1.899781 

Epoch 3
-------------------------------
loss: 1.923641  [    0/60000]
loss: 1.892891  [ 6400/60000]
loss: 1.784286  [12800/60000]
loss: 1.821564  [19200/60000]
loss: 1.731934  [25600/60000]
loss: 1.670496  [32000/600

In [15]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
